In [3]:
import pandas as pd
import numpy as np
import math

%matplotlib inline

#### All data given as of 8/28/2004

In [12]:
cap_rate = 0.0475      # given in HW2_Data file
short_rate = 0.01816   # given

df_stmat = pd.read_csv('stmat.csv')
df_tvmat = pd.read_csv('tvmat.csv')
df_disc_factors = pd.read_csv('discount factors.csv')
df_caplet_vols = pd.read_csv('caplet vols.csv')
df_mthly_10_yr_LIBOR_04 = pd.read_csv('monthly 10 year LIBOR spot rates for 2004.csv')

df_stmat['Coupon Gap'] = df_stmat['Coupon Gap'] * 0.0001 # convert from basis points to decimal
df_tvmat['Coupon Gap'] = df_tvmat['Coupon Gap'] * 0.0001 # convert from basis points to decimal

#### Ouput results from matlab functions

In [16]:
gamma = 0.0568
p = 2.9025
beta_1 = -12.6051
beta_2 = -2.0992

#### Standard error results from matlab code

In [17]:
gamma_se = 0.0018
p_se = 0.0750
beta_1_se = 1.9998
beta_2_se = 0.0495

# Q1

#### hazard function

SyntaxError: invalid syntax (<ipython-input-15-04124acd8bd0>, line 6)